In [1]:
import numpy as np
import galois
import random
from py_ecc.bn128 import G1,G2,multiply, add, curve_order, eq, Z1 


#=========================QAP======================================
U=[
    [2, 2, 1, 2],#x^3 + 2x^2 + x + 2
    [0, 0, 0, 0],#0
    [4, 5, 5, 1],#4x^3 + 5x^2 + 5x + 1
    [4, 3, 6, 1],#4x^3 + 3x^2 + 6x + 1
    [3, 0, 0, 4],#3x^3 + 4
    [6, 6, 3, 6]#6x^3 + 6x^2 + 3x + 6
]

V=[
    [2, 6, 3, 3],#2x^3 + 6x^2 + 3x + 3
    [0, 0, 0, 0],#0
    [5, 1, 4, 5],#5x^3 + x^2 + 4x + 5
    [0, 0, 0, 0],#0
    [0, 0, 0, 0],#0
    [0, 0, 0, 0]#0
]

W=[
    [0, 0, 0, 0],#0
    [6, 6, 3, 6],#6x^3 + 6x^2 + 3x + 6
    [0, 0, 0, 0],#0
    [1, 5, 5, 4],#x^3 + 5x^2 + 5x + 4
    [4, 3, 6, 1],#4x^3 + 3x^2 + 6x + 1
    [3, 0, 0, 4]#3x^3 + 4
]

t=[1, 4, 0, 6, 3]

h=[4, 3, 2]

a=[1, 135, 5, 25, 125, 130]

nb_var=len(U[0])
nb_equation=len(U)

print("nb_var: ",nb_var)
print("nb_equation: ",nb_equation)


#curve_order=7 #prenons 7
tau=random.randint(1,curve_order)
print("tau: ",tau)

vectG1=[]
vectG2=[]
for i in range(nb_var):
    vectG1.append(multiply(G1,tau**i))
    vectG2.append(multiply(G2,tau**i))

print("vectG1: ",vectG1)
print("vectG2: ",vectG2)


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


nb_var:  4
nb_equation:  6
tau:  10246650909060523011744742479254081366669408523347358602227464273959127073611
vectG1:  [(1, 2), (7566475444277910843221594733320799714844148738693633505018246460070427107952, 21283664570385309555768851468640751791903200545233956737135199499053997864711), (21551663782423110929533489798826920549716424023021440201180173594019508893155, 20072245734975882557539065708952947022422471940014658961182783986612557790747), (7620914374586239297388311232477553332112036198582476797242717816858013308612, 11135534076714506462411421998343091206743601449374892186349783431317798932123)]
vectG2:  [((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531)), ((9539298877834327768178851465285618214252116215922726014463222

In [2]:
def eval_polynomial_desc(poly, x):
    result = 0
    # L'ordre des puissances est décroissant, donc on commence avec la puissance la plus élevée
    power = len(poly) - 1
    for coef in poly:
        result += coef * (x ** power)
        power -= 1  # Diminuer la puissance pour l'itération suivante
    return result

In [5]:
#coté du prover
from py_ecc.bn128 import pairing, FQ

#calcul de [A]_1

A1=Z1
for i in range(nb_equation):
    sum_inner=Z1#neutre

    for j in range(nb_var):
        sum_inner=add(sum_inner,multiply(vectG1[j],U[i][j]))
        #print(f"temp {i},{j}: ",sum_inner)

    A1=add(A1,multiply(sum_inner,a[i]))
    #print("A1: ",A1)
    
print("A1: ",A1)
#print("G1:",G1)

#calcul de [B]_2

B2=Z1
for i in range(nb_equation):
    sum_inner=Z1#neutre

    for j in range(nb_var):
        sum_inner=add(sum_inner,multiply(vectG2[j],V[i][j]))
        #print(f"temp {i},{j}: ",sum_inner)

    B2=add(B2,multiply(sum_inner,a[i]))
    #print("B2: ",B2)

print("B2: ",B2)


#calcul de [C]_1

C1_1=Z1
for i in range(nb_equation):
    sum_inner=Z1#neutre

    for j in range(nb_var):
        sum_inner=add(sum_inner,multiply(vectG1[j],W[i][j]))
        #print(f"temp {i},{j}: ",sum_inner)

    C1_1=add(C1_1,multiply(sum_inner,a[i]))
    #print("C1: ",C1)

C1_2=Z1
t_tau=eval_polynomial_desc(t, tau)
for i in range(len(h)):
    C1_2=add(C1_2,multiply(multiply(vectG1[i],t_tau),h[i]))
    #print("C1: ",C1)

#C1=C1_1+C1_2
C1=C1_1


print("C1: ",C1)

print("\nG1:",G1)
print("G2:",G2)

print("les types de A B C G1 G2: ",type(A1[1]),type(B2),type(C1),type(G1),type(G2))

print(pairing(B2, A1) == pairing(G2, C1))


A1:  (5840311249393096865063288678248730391909192108729918293992857658207897122303, 8529205685888835183168023880687086675478731454692804563270175773437821766899)
B2:  ((4157239275812761375439519701528760855476623033879092869293828678539200743893, 12648418490899446575998329124722545992046144941787274586450571320479977680422), (6757545376023035940980029121652429431215873318080791444028669991628630537889, 3872337090059641834574525689658602163125798871224586737995377414142455995817))
C1:  (10170119537082865328545732144619398058343281226714029841146768853935302552976, 14744452753352212882366187685706727336464396567787522408031888528681870005051)

G1: (1, 2)
G2: ((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531))
les types de A B